<a href="https://colab.research.google.com/github/gabrimatx/Jupyter-Notebooks/blob/main/CUDA_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CUDA Notebook

In [26]:
!nvcc --version
!nvidia-smi
!nvidia-smi --query-gpu=compute_cap --format=csv
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Sun Jan 28 23:14:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  7

## CUDA: Compute Unified Device Architecture
- Enables explicit GPU memory management.
- GPU is viewed as *compute device* and is a co-processor to the CPU, has its own **DRAM** and runs many threads in parallel.
---
- The GPU architecture is built around a scalable array of streaming multiprocessors (SM).
- Every SM in a GPU support the concurrent execution of hundreds of threads.
- Every SM executes thread in groups of 32 called **warps**. All threads in a warp execute the same instruction at the same time.

# Typical structure of a CUDA program
- GPU memory allocation.
- Transfer data to CPU.
- Execute CUDA kernel.
- Copy results from GPU's memory.
- Reinitialize device.

# Hello world

In [25]:
%%cuda
#include <stdio.h>

__global__ void hello(){
    printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main(){
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
}

Hello from block: 1, thread: 0
Hello from block: 1, thread: 1
Hello from block: 0, thread: 0
Hello from block: 0, thread: 1



# Function decorations
`__device__` : a function that can only be called from within a kernel, i.e. not from the host.  

`__host__` : a function that can only run on the host. The __host__ qualifier is typically omitted, unless used in combination with __device__ to indicate that the function can run on both the host and the device. Such a scenario implies the generation of two compiled codes for the function.

`__global__` : a function that can be called from the host and executed on the device. In CC 3.5 and above, the device can also call __global__ functions, using a feature called dynamic parallelism.

# CUDA Memory Management Functions

- `cudaMalloc(address, size)`: allocates object in device global memory.
- `cudaFree(address)`: frees object from device global memory.
- `cudaMemcpy(destination, source, bytes_copied, direction_of_transfer)`: memory data transfer, synchronous.

# Vector addition

In [67]:
%%cuda
#include <stdio.h>
#include <cuda.h>
__global__ void vecAddKernel(float *A_d, float *B_d, float *C_d, int n){
    int i = threadIdx.x + blockDim.x * blockIdx.x;
    if (i < n) C_d[i] = A_d[i] + B_d[i];
}

void vecAdd(float *A_h, float *B_h, float *C_h, int n){
    int size  = n * sizeof(float);
    float *A_d, *B_d, *C_d;




    // Transfer A and B to device memory
    cudaMalloc(&A_d, size);
    cudaMemcpy(A_d, A_h, size, cudaMemcpyHostToDevice);
    cudaMalloc(&B_d, size);
    cudaMemcpy(B_d, B_h, size, cudaMemcpyHostToDevice);

    // Allocate device memory for result
    cudaMalloc(&C_d, size);

    // Kernel invocation
    vecAddKernel<<<ceil(n/256.0),256>>>(A_d, B_d, C_d, n);
    cudaDeviceSynchronize();



    // Transfer C to host
    cudaMemcpy(C_h, C_d, size, cudaMemcpyDeviceToHost);
    printf("C = [%f %f %f]", C_h[0], C_h[1], C_h[2]);

    // Free device memory
    cudaFree(A_d); cudaFree(B_d); cudaFree(C_d);

}

int main(){
    float A[] = {1,2,3}, B[] = {1,2,3}, C[3];
    vecAdd(A, B, C, 3);
    return 0;
}

C = [2.000000 4.000000 6.000000]


# Kernel restrictions
- Can only access device memory.
- Must be of void type.
- Does not support a variable number of arguments.
- Does not support static variables.
- Does not support pointers to functions.
- Is async w.r.t. the host.

# Intrinsic variables
- `blockDim`: Contains the size of each block e.g. $(B_x, B_y, B_z)$.
- `gridDim`: Contains the size of the grid in blocks e.g. $(G_x, G_y, G_z)$.
- `threadIdx`: The $(x,y,z)$ position of the thread within a block, with $x\in [0, B_x-1],y\in [0,B_y - 1], z\in [0, B_z-1]$.
- `blockIdx`: The $(b_x,b_y,b_z)$ position of thread's block within the grid, with $b_x\in [0, G_x-1],b_y\in [0,G_y - 1], b_z\in [0, G_z-1]$.

`blockDim` and `gridDim` are of type `dim3` a type of integer vectors based on `uint3`. When a variable of type `dim3` is initialized, non defined components are set to 1. Components in a `dim3` variable are accessible with `x,y,z`.
